In [7]:
from __future__ import print_function
import sys
import requests
import json
import tweepy
import pickle

### IBM Personality API Keys

In [8]:
pi_url = "https://gateway.watsonplatform.net/personality-insights/api"
pi_username = "#######"
pi_password = "#######"

In [9]:
def ibm_personality_api(pi_content_items):
    # { "url": "https://gateway.watsonplatform.net/personality-insights/api", "username": "#######", "password": "#####" }
    r = requests.post(pi_url + '/v2/profile',
                      auth=(pi_username, pi_password),
                      headers={
                          'content-type': 'application/json',
                          'accept': 'application/json'
                      },
                      data=json.dumps(pi_content_items)
                      )

    print("Profile Request sent. Status code: %d, content-type: %s" %
          (r.status_code, r.headers['content-type']))
    personality_output = json.loads(r.text)
    return personality_output

In [10]:
def convert_status_to_pi_content_item(names, tweet_text):
    # My code here
    return {
        'userid': names,
#         'id': names+'_test',
        'sourceid': 'python-twitter',
        'contenttype': 'text/plain',
#         'language': s.lang,
        'content': tweet_text,
        # 'created': s.created_at,
#         'reply': (s.in_reply_to_status_id is None),
#         'forward': False
    }

In [4]:
# Getting the required files :
with open('Refined_Project_Dataset/all_user_tweets.pkl','rb') as fp:
    all_user_tweets = pickle.load(fp)

In [ ]:
with open('Refined_Project_Dataset/user_personality.pkl','rb') as fp:
    final_user_personality = pickle.load(fp)

existing_user_personality = set()
for each in final_user_personality:
    try:
        existing_user_personality.add(each['id'])
    except:
        print('missing user')
        continue

    
print(len(existing_user_personality))

for names,tweets in all_user_tweets.items():
    if names in existing_user_personality:
        continue
    print(names)
    pi_content_items_array = []
    for each_tweet in tweets:
        pi_content_items_array.append(convert_status_to_pi_content_item(names,each_tweet))
    
    pi_content_items = {'contentItems': pi_content_items_array}
    personality_output = ibm_personality_api(pi_content_items)
    final_user_personality.append(personality_output)

In [23]:
temp = final_user_personality
len(temp)

2668

In [22]:
# with open('Refined_Project_Dataset/user_personality.pkl','wb') as fp:
#     pickle.dump(final_user_personality,fp)
with open('Refined_Project_Dataset/user_personality.pkl','rb') as fp:
    final_user_personality = pickle.load(fp)

In [19]:
import pandas as pd
df = pd.read_json('Refined_Project_Dataset/Binny_Tagged_Final.json')
hate_usernames = set(df['original_tweetusername'].values.tolist())
counter_usernames = set(df[df.tag!='0']['replier_user_name'].values.tolist())
Non_counter_usernames = set(df[df.tag=='0']['replier_user_name'].values.tolist())
len(hate_usernames),len(counter_usernames),len(Non_counter_usernames)

(548, 1239, 220)

In [27]:
final_user_personality[2665]

{'id': 'RoyDunn83494681',
 'source': 'python-twitter',
 'word_count': 347,
 'word_count_message': 'There were 347 words in the input. We need a minimum of 600, preferably 1,200 or more, to compute statistically significant estimates',
 'processed_lang': 'en',
 'tree': {'id': 'r',
  'name': 'root',
  'children': [{'id': 'personality',
    'name': 'Big 5',
    'children': [{'id': 'Extraversion_parent',
      'name': 'Extraversion',
      'category': 'personality',
      'percentage': 0.9519702960548467,
      'children': [{'id': 'Openness',
        'name': 'Openness',
        'category': 'personality',
        'percentage': 0.8550937559093148,
        'sampling_error': 0.0627462137,
        'children': [{'id': 'Adventurousness',
          'name': 'Adventurousness',
          'category': 'personality',
          'percentage': 0.7133808254951195,
          'sampling_error': 0.052771050400000005},
         {'id': 'Artistic interests',
          'name': 'Artistic interests',
          'categ

In [31]:
Hate_user_personality = []
Counter_user_personality = []
Noncounter_user_personality = []
for each_personality in final_user_personality:
    try:
        if each_personality['id'] in hate_usernames:
            Hate_user_personality.append(each_personality)
    except:
        pass
    
    try:
        if each_personality['id'] in counter_usernames:
            Counter_user_personality.append(each_personality)
    except:
        pass
    
    try:
        if each_personality['id'] in Non_counter_usernames:
            Noncounter_user_personality.append(each_personality)
    except:
        pass

In [32]:
len(Hate_user_personality),len(Counter_user_personality),len(Noncounter_user_personality)

(505, 1160, 204)

In [33]:
with open('Refined_Project_Dataset/Hate_user_personality.pkl','wb') as fp:
    pickle.dump(Hate_user_personality,fp)
with open('Refined_Project_Dataset/Counter_user_personality.pkl','wb') as fp:
    pickle.dump(Counter_user_personality,fp)
with open('Refined_Project_Dataset/Noncounter_user_personality.pkl','wb') as fp:
    pickle.dump(Noncounter_user_personality,fp)